In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

import matplotlib.pyplot as plt

The IMDB large movie review dataset is a binary classification dataset—all the reviews have either a positive or negative sentiment.

Download the dataset using TFDS.

In [ ]:
dataset, info =  tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteVYXI21/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteVYXI21/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteVYXI21/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [ ]:
# split train and test dataset

train_dataset, test_dataset = dataset['train'], dataset['test']

In [ ]:
encoder = info.features['text'].encoder

print('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

# Prepare the data for training

In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE)

test_dataset = test_dataset.padded_batch(BATCH_SIZE)

# expand dimension 
train_dataset = train_dataset.map(lambda x_text, x_label: (x_text, tf.expand_dims(x_label, -1)))
test_dataset = test_dataset.map(lambda x_text, x_label: (x_text, tf.expand_dims(x_label, -1)))

# Create the model

In [ ]:
model = tf.keras.Sequential([
                     tf.keras.layers.Embedding(encoder.vocab_size, 64),
                     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                     tf.keras.layers.Dense(64, activation='relu'),
                     tf.keras.layers.Dense(32, activation='relu'),
                     tf.keras.layers.Dense(1)
])

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 64)          523840    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 600,257
Trainable params: 600,257
Non-trainable params: 0
_________________________________________________________________


Compile the Keras model to configure the training process:

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics='accuracy')

# Train the model

In [ ]:
mode_history = model.fit(train_dataset, epochs=10, validation_steps=30, validation_data=test_dataset)

Epoch 1/10
391/391 [==============================] - 44s 113ms/step - loss: 0.6496 - accuracy: 0.5543 - val_loss: 0.4490 - val_accuracy: 0.7734
Epoch 2/10
391/391 [==============================] - 43s 111ms/step - loss: 0.3331 - accuracy: 0.8616 - val_loss: 0.3502 - val_accuracy: 0.8323
Epoch 3/10
391/391 [==============================] - 44s 112ms/step - loss: 0.2446 - accuracy: 0.9068 - val_loss: 0.3306 - val_accuracy: 0.8766
Epoch 4/10
391/391 [==============================] - 44s 112ms/step - loss: 0.2061 - accuracy: 0.9251 - val_loss: 0.3508 - val_accuracy: 0.8578
Epoch 5/10
391/391 [==============================] - 44s 112ms/step - loss: 0.1742 - accuracy: 0.9392 - val_loss: 0.3551 - val_accuracy: 0.8677
Epoch 6/10
391/391 [==============================] - 44s 112ms/step - loss: 0.1542 - accuracy: 0.9473 - val_loss: 0.3573 - val_accuracy: 0.8661
Epoch 7/10
391/391 [==============================] - 43s 111ms/step - loss: 0.1423 - accuracy: 0.9508 - val_loss: 0.3830 - val_ac

# Evaluate model

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

391/391 [==============================] - 19s 49ms/step - loss: 0.4680 - accuracy: 0.8533
Test Loss: 0.4679550528526306
Test Accuracy: 0.8533200025558472
